# modular analysis of DeepLabCut pose estimations
## test_extract_params_from_dlc.ipynb

Analyze .h5 files output by DeepLabCut and Anipose and extract properties of mice (and prey if present). Outputs of this pipeline are saved out as .nc files and can be opened as xarrays. Run this in the ```fmephys``` environment.

Last modified September 14, 2020

In [ ]:
import json
import xarray
from tqdm.notebook import trange, tqdm
from nb_wrapper import topdown_intake, eye_intake
from util.save_data import savetrial


## user inputs

In [ ]:
json_config_path = '/path/to/pipeline_config.json'
trial_name = '010101_mouse1_cricket1'

### open config file

In [ ]:
with open(json_config_path, 'r') as fp:
    config = json.load(fp)
config

### top camera

In [ ]:
%%time

xTop = topdown_intake(config, trial_name)

for num in range(1,toptrial_count+1):
    trialext = 'TOP' + str(num)
    top_trialname = trial_name + '_' + trialext
    xtop = topdown_intake(global_data_path, top_trialname, trialext, global_save_path, lik_thresh, coord_cor, topdown_pt_num, cricket, bonsaitime)
    savetrial(xtop, global_save_path, trial_name, trialext)


## eye tracking and worldcam adjustment

### left eye

In [ ]:
%%time
LEye = trial_name + '_LEYE'

# find ellipse parameters
xLEye = eye_intake(global_data_path, LEye, 'LEYE', global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)
savetrial(xLEye, global_save_path, trial_name, 'LEYE')


### right eye

In [ ]:
%%time
REye = trial_name + '_REYE'

# find ellipse parameters
xREye, rfit, shift = eye_intake(global_data_path, REye, 'REYE', global_save_path, lik_thresh, pxl_thresh, ell_thresh, eye_pt_num, tear, bonsaitime)
savetrial(xREye, global_save_path, trial_name, 'REYE')
